##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introducción a TensorBoard

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tensorboard/get_started"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tensorboard/get_started.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tensorboard/get_started.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tensorboard/get_started.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

En el aprendizaje automático, para mejorar algo frecuentemente es necesario poder medirlo. TensorBoard es una herramienta que proporciona las medidas y visualizaciones necesarias durante el flujo de trabajo del aprendizaje automático. Permite realizar el seguimiento de métricas de experimentos como la pérdida y la precisión, visualizar el gráfico del modelo, proyectar incrustaciones a un espacio de menor dimensión y mucho más.

Esta guía rápida le mostrará cómo iniciarse rápidamente con TensorBoard. Las guías restantes de este sitio web proporcionan más detalles sobre capacidades específicas, muchas de las cuales no se incluyen aquí. 

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

Utilizando el conjunto de datos [MNIST](https://en.wikipedia.org/wiki/MNIST_database) como ejemplo, normalice los datos y escriba una función que cree un modelo Keras sencillo para clasificar las imágenes en 10 clases.

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

11493376/11490434 [==============================] - 0s 0us/step


## Cómo utilizar TensorBoard con Keras Model.fit()

Cuando se entrena con [Model.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit) de Keras, si se agrega la retrollamada `tf.keras.callbacks.TensorBoard` se asegura de que los registros se crean y almacenan. Además, active el cálculo del histograma en cada época con `histogram_freq=1` (está desactivado de forma predeterminada).

Coloque los registros en un subdirectorio con marca de tiempo para permitir una selección fácil de las diferentes corridas del entrenamiento.

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 15s 246us/sample - loss: 0.2217 - accuracy: 0.9343 - val_loss: 0.1019 - val_accuracy: 0.9685
Epoch 2/5
60000/60000 [==============================] - 14s 229us/sample - loss: 0.0975 - accuracy: 0.9698 - val_loss: 0.0787 - val_accuracy: 0.9758
Epoch 3/5
60000/60000 [==============================] - 14s 231us/sample - loss: 0.0718 - accuracy: 0.9771 - val_loss: 0.0698 - val_accuracy: 0.9781
Epoch 4/5
60000/60000 [==============================] - 14s 227us/sample - loss: 0.0540 - accuracy: 0.9820 - val_loss: 0.0685 - val_accuracy: 0.9795
Epoch 5/5
60000/60000 [==============================] - 14s 228us/sample - loss: 0.0433 - accuracy: 0.9862 - val_loss: 0.0623 - val_accuracy: 0.9823


Inicie TensorBoard mediante la línea de comandos o dentro de una experiencia del bloc de notas. Las dos interfaces son generalmente las mismas. En los cuadernos, utilice la magia de la línea `%tensorboard`. En la línea de comandos, ejecute el mismo comando sin "%".

In [ ]:
%tensorboard --logdir logs/fit

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_model_fit.png?raw=1"/> -->

Un breve resumen de las visualizaciones creadas en este ejemplo y los paneles de control (pestañas en la barra de navegación superior) donde se pueden encontrar:

- Los **Escalares** muestran cómo cambian la pérdida y las métricas con cada época. Puede utilizarlos para controlar también la velocidad de entrenamiento, la tasa de aprendizaje y otros valores escalares. Los escalares pueden encontrarse en los paneles **Serie temporal** o **Escalares**.
- Los **Gráficos** le ayudan a visualizar su modelo. En este caso, se muestra el gráfico de capas de Keras, que puede ayudarle a asegurarse de que está construido correctamente. Los gráficos se encuentran en el panel **Gráficos**.
- **Histogramas** y **Distribuciones** muestran la distribución de un Tensor a lo largo del tiempo. Esto puede ser útil para visualizar las ponderaciones y los sesgos y verificar que están cambiando de la forma esperada. Los histogramas pueden encontrarse en los paneles de control **Series temporales** o **Histogramas**. Las distribuciones pueden encontrarse en el panel **Distribuciones**.

Los paneles de control adicionales de TensorBoard se habilitan automáticamente cuando se registran otros tipos de datos. Por ejemplo, la retrollamada de Keras TensorBoard le permite registrar también imágenes e incrustaciones. Puede ver qué otros paneles de control están disponibles en TensorBoard haciendo clic en el menú desplegable "inactivo" situado en la parte superior derecha.

## Cómo utilizar TensorBoard con otros métodos


Cuando entrene con métodos como [`tf.GradientTape()`](https://www.tensorflow.org/api_docs/python/tf/GradientTape), utilice `tf.summary` para registrar la información necesaria.

Utilice el mismo conjunto de datos que en el caso anterior, pero conviértalo en `tf.data.Dataset` para aprovechar las capacidades de procesamiento por lotes:

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

El código de entrenamiento sigue el tutorial sobre el [arranque rápido avanzado](https://www.tensorflow.org/tutorials/quickstart/advanced), pero muestra cómo registrar las métricas en TensorBoard. Elija la pérdida y el optimizador:

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

Cree métricas de estado que puedan utilizarse para acumular valores durante el entrenamiento y registrarse en cualquier momento:

In [ ]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

Defina las funciones de entrenamiento y de prueba:

In [ ]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

Configure los escritores de resúmenes para que escriban los resúmenes en el disco con un directorio de registros diferente:

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

Inicie el entrenamiento. Utilice `tf.summary.scalar()` para registrar las métricas (pérdida y precisión) durante el entrenamiento/prueba en el ámbito de los escritores de resúmenes para escribir los resúmenes en el disco. Usted tiene el control sobre qué métricas registrar y con qué frecuencia hacerlo. Otras funciones `tf.summary` permiten registrar otros tipos de datos.

In [ ]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.24321186542510986, Accuracy: 92.84333801269531, Test Loss: 0.13006582856178284, Test Accuracy: 95.9000015258789
Epoch 2, Loss: 0.10446818172931671, Accuracy: 96.84833526611328, Test Loss: 0.08867532759904861, Test Accuracy: 97.1199951171875
Epoch 3, Loss: 0.07096975296735764, Accuracy: 97.80166625976562, Test Loss: 0.07875105738639832, Test Accuracy: 97.48999786376953
Epoch 4, Loss: 0.05380449816584587, Accuracy: 98.34166717529297, Test Loss: 0.07712937891483307, Test Accuracy: 97.56999969482422
Epoch 5, Loss: 0.041443776339292526, Accuracy: 98.71833038330078, Test Loss: 0.07514958828687668, Test Accuracy: 97.5


Abra de nuevo TensorBoard, esta vez dirigiéndolo hacia el nuevo directorio de registro. También podríamos haber iniciado TensorBoard para monitorizar el entrenamiento mientras progresa.

In [ ]:
%tensorboard --logdir logs/gradient_tape

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_gradient_tape.png?raw=1"/> -->

Ya está. Ahora han visto cómo utilizar TensorBoard tanto mediante la retrollamada de Keras como por medio de `tf.summary` para escenarios más personalizados. 

## TensorBoard.dev: Aloje y comparta los resultados de sus experimentos de ML

[TensorBoard.dev](https://tensorboard.dev) es un servicio público gratuito que le permite subir sus registros de TensorBoard y obtener un permalink que puede ser compartido con todo el mundo en trabajos académicos, entradas de blog, medios sociales, etc.  Esto puede permitir una mejor reproducibilidad y colaboración.

Para utilizar TensorBoard.dev, ejecute el siguiente comando:


In [ ]:
!tensorboard dev upload \
  --logdir logs/fit \
  --name "(optional) My latest experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \
  --one_shot

Tenga en cuenta que esta invocación utiliza el prefijo de exclamación (`!`) para invocar el intérprete de comandos en vez del prefijo de porcentaje (`%`) para invocar la magia de colab.  Al invocar este comando desde la línea de comandos no es necesario ninguno de los dos prefijos.

Vea un ejemplo [aquí](https://tensorboard.dev/experiment/EDZb7XgKSBKo6Gznh3i8hg/#scalars).

Para obtener más detalles sobre cómo utilizar TensorBoard.dev, consulte https://tensorboard.dev/#get-started.